In [1]:
import pymaid
source = pymaid.CatmaidInstance('https://fafb.catmaid.virtualflybrain.org/' , api_token=None)

neurons = pymaid.get_annotated("Paper: Zheng et al 2018") # Paper of interest

INFO  : Global CATMAID instance set. Caching is ON. (pymaid)
INFO  : Searching for: Paper: Zheng et al 2018 (pymaid)


### This step will possibly throw an error depending on the version of pymaid you have. We tested this with the latest version of pymaid from Github, which you can download with `pip3 install git+git://github.com/schlegelp/pymaid@master`

## Get Neuron IDs in a List for Indexing

In [2]:
neuron_ids = [j for j in neurons["skeleton_ids"]] # Get neuron IDs

old = neuron_ids
neuron_ids = []

for i in range(len(old)):
    neuron_ids.append(old[i][0])

## Import Connectome

In [3]:
#pymaid.get_partners(neuron_ids)
#pymaid.cn_table_from_connectors(neuron_ids)

metadata_dict = pymaid.get_annotations(neuron_ids)

In [4]:
import matplotlib.pyplot as plt

connectome = pymaid.adjacency_matrix(neuron_ids)
#connectome = connectome.to_numpy()

#from graspologic.plot import heatmap
#heatmap(connectome)

#plt.show()

In [6]:
from grapgh_io import GraphIO
import networkx as nx

connectome_nx = nx.from_pandas_adjacency(connectome)

In [24]:
for i in list(connectome_nx.nodes):
    connectome_nx.nodes[i]['annotation'] = metadata_dict["{}".format(i)]

G = GraphIO.dump(connectome_nx, "./fafb.json")